# Agent-based bacteria colony simulation

In [1]:
# To - do

# multiplication of bacteria
# eating food by bacteria
# chart with bacteria growth with time
# random moving of bacteria

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import random
from IPython.display import HTML

In [3]:
class Bacterium:
    def __init__(self, state):  # 0 - empty cell, 1 - young, 2 - ready to divide, 3 - dead
        self.state = state
        self.hunger_level = 0 # if hunger_level == 10 -> dies
        self.can_use_galactose = 0
        self.maturity = 0
        
    def get_hungry(self, speed=1):
        self.hunger_level += 1*speed
    
    def learn_to_use_galactose(self):
        if self.hunger_level > 6:
            self.can_use_galactose = 1
            
    def mature(self):
        if self.hunger_level < 5:
            self.maturity +=1
            
    def check_if_ready_to_divide(self, is_bacteriostatic):
        if self.maturity > 4 and is_bacteriostatic == 0:
            self.state = 2
            
    def after_division(self): # we use it for 2 cells that are created after division
        self.state = 1
        self.hunger_level = 2
        self.can_use_galactose = 0
        self.maturity = 0
    
    def check_if_die(self, is_antibiotic):
        if is_antibiotic == 1 or self.hunger_level == 10:
            self.state = 3 # bacteria is dead
    
    def getColour(self):
        if self.state == 0:   
            return 0
        elif self.state == 1:
            return 255
        elif self.state == 2:
            return 200
        else:
            return 150  # for dead cells
    
    def move(self):
        return

    

In [4]:
class Food:
    def __init__(self, FoodType):
        self.FoodType = FoodType   # 1 - glucose, 2 - galactose, 0 - no food
        if FoodType == 1:
          self.Reserve = 2
        elif FoodType == 2:
          self.Reserve = 4
        else:
          self.Reserve = 0 

    def getColour(self):
        if self.FoodType == 1:   # glucose - light blue
            return "lightblue"
        elif self.FoodType == 2:   # galactose - dark blue
            return "darkblue"

    def check_reserve(self):
        self.Reserve -= 1
        if self.Reserve == 0:
          self.FoodType = 0

    def __repr__(self):
        return str(self.FoodType)
        

In [5]:
class Board:
    
    def __init__(self, xSize, ySize):    
        self.xSize = xSize
        self.ySize = ySize
        self.bacteria_cells = []
        self.food_cells = []
             
        for i in range(0, xSize):
            b_row = []
            f_row = []
            for j in range(0, ySize):
                b_row.append(Bacterium(0))
                f_row.append(Food(0))
            self.bacteria_cells.append(b_row)
            self.food_cells.append(f_row)
        
        self.randomBoard()

               
    def printBoard(self):
        print('printing the board')
        print(self.cells)
                                                    
    def randomBoard(self):  #TODO - initializing a random board with food

        self.food_cells[1][4] = Food(1)
        self.food_cells[1][2] = Food(1)
        self.food_cells[2][2] = Food(1)
        self.food_cells[1][3] = Food(2)

        self.food_cells[5][4] = Food(2)
        self.food_cells[5][5] = Food(2)
        self.food_cells[6][5] = Food(2)
        
        self.bacteria_cells[1][2] = Bacterium(1)
        self.bacteria_cells[5][1] = Bacterium(1)
        self.bacteria_cells[5][5] = Bacterium(1)

        # random boolean mask    
        """
        fields_mask = np.random.choice([0,1,2,3],size=(self.xSize,self.ySize), p=[0.3,0.2,0.2,0.3])

        for i in range (0,len(fields_mask)-1):
            for j in range (0,len(fields_mask[0])-1):
                if fields_mask[i][j] == 1:
                    self.cells[i][j] = Food(1)
                elif fields_mask[i][j] == 2:
                    self.cells[i][j] = Food(2)
                elif fields_mask[i][j] == 3:
                    self.cells[i][j] = Bacterium()
        """            
        return

        
    def iteration(self): 
        print('iteration!')
        #self.printBoard()
        for i in range(1,len(self.bacteria_cells)-1):
            for j in range(1, len(self.bacteria_cells[0])-1):
                # Moore
                neigh_cells = [[i-1,j-1], [i-1,j], [i-1,j+1],
                                  [i,j-1], [i,j+1], [i+1,j-1], 
                                  [i+1,j], [i+1,j+1]]
                if self.bacteria_cells[i][j].state == 2:
                        self.bacteria_cells[i][j].after_division()
                        move = random.choice(neigh_cells)
                        while move[0] == 0 or move[0] == len(self.bacteria_cells)-1 or move[1] == 0 or move[1] == len(self.bacteria_cells[0])-1 and self.bacteria_cells[i][j].state != 0:
                            move = random.choice(neigh_cells)
                        self.bacteria_cells[move[0]][move[1]].after_division()
                elif self.bacteria_cells[i][j].state == 1:
                    move = random.choice(neigh_cells)
                    print(move)
                    while move[0] == 0 or move[0] == len(self.bacteria_cells)-1 or move[1] == 0 or move[1] == len(self.bacteria_cells[0])-1:
                        move = random.choice(neigh_cells)
                    x,y = i,j
                    if self.food_cells[i][j].FoodType == 1:
                          print('food1 for:', i,j, 'state',self.bacteria_cells[i][j].state)
                          self.bacteria_cells[i][j].hunger_level -= 2
                          self.food_cells[i][j].FoodType = 0
                    elif self.food_cells[i][j].FoodType == 2 and self.bacteria_cells[i][j].can_use_galactose != 0:
                          print('food2 for:', i,j, 'state',self.bacteria_cells[i][j].state)
                          self.bacteria_cells[i][j].hunger_level -= 1
                          self.food_cells[i][j].FoodType = 0
                    else:
                        if self.bacteria_cells[move[0]][move[1]].state == 0:
                            x,y = move[0],move[1]
                            self.bacteria_cells[move[0]][move[1]].state = self.bacteria_cells[i][j].state
                            self.bacteria_cells[move[0]][move[1]].hunger_level = self.bacteria_cells[i][j].hunger_level
                            self.bacteria_cells[move[0]][move[1]].can_use_galactose = self.bacteria_cells[i][j].can_use_galactose
                            self.bacteria_cells[move[0]][move[1]].maturity = self.bacteria_cells[i][j].maturity
                            self.bacteria_cells[i][j].state = 0

                            if self.food_cells[move[0]][move[1]].FoodType == 1:
                                print('food11 for:', i,j, 'state',self.bacteria_cells[i][j].state)
                                self.bacteria_cells[move[0]][move[1]].hunger_level -= 2
                                self.food_cells[move[0]][move[1]].FoodType = 0
                            elif self.food_cells[move[0]][move[1]].FoodType == 2 and self.bacteria_cells[move[0]][move[1]].can_use_galactose != 0:
                                print('food22 for:', i,j, 'state',self.bacteria_cells[i][j].state)
                                self.bacteria_cells[move[0]][move[1]].hunger_level -= 1
                                self.food_cells[move[0]][move[1]].FoodType = 0
                            else:
                                self.bacteria_cells[move[0]][move[1]].get_hungry()
                        else:
                            self.bacteria_cells[i][j].get_hungry()
                    self.bacteria_cells[x][y].mature()
                    self.bacteria_cells[x][y].learn_to_use_galactose()
                    self.bacteria_cells[x][y].check_if_die(is_antibiotic=0)
                    self.bacteria_cells[x][y].check_if_ready_to_divide(is_bacteriostatic=0)
                elif self.bacteria_cells[i][j].state == 3:
                    self.bacteria_cells[i][j] = Bacterium(0)


                    #print(self.food_cells)

        

    def clear(self):
        """
        for i in range(0,len(self.food_items)):
            for j in range(0, len(self.food_items[0])):
                self.food_items[i][j].clear_static()
        #print('robie clear booard')
        self.calculateField()
        """
    
    def redraw_bacteria(self):
        # Returns numpy array of colours
        print('redraws!')
        fig = np.zeros((len(self.bacteria_cells),len(self.bacteria_cells[0])))
        for i in range(0,len(self.bacteria_cells)):
            for j in range(0, len(self.bacteria_cells[0])):
                fig[i][j] = self.bacteria_cells[i][j].getColour()
        #print('fig:', fig)
        return fig
    
    def redraw_food(self):
        # Returns positions of glucose
        glucose_x = []
        glucose_y = []
        galactose_x = []
        galactose_y = []
        for i in range(0,len(self.food_cells)):
            for j in range(0, len(self.food_cells[0])):
                if self.food_cells[i][j].FoodType == 1:
                    glucose_x.append(i)
                    glucose_y.append(j)
                elif self.food_cells[i][j].FoodType == 2:
                    galactose_x.append(i)
                    galactose_y.append(j)
        #indexes = np.where(self.food_cells.FoodType == 1) 
        #print(glucose_x, glucose_y, galactose_x, galactose_y)
        return glucose_x, glucose_y, galactose_x, galactose_y
                
            

In [6]:
def update(frameNum, board, ax):
    if(frameNum == 0):
        board.randomBoard()       
    else:   
        board.iteration()
        ax.clear()
    #im.set_data(board.redraw_bacteria())

    im = ax.imshow(board.redraw_bacteria(),  vmin=0, vmax=255)
    glucose_x, glucose_y, galactose_x, galactose_y = board.redraw_food()
    sct_glucose = ax.scatter(glucose_y, glucose_x, c='lightblue', marker='+', s=150)
    sct_galactose = ax.scatter(galactose_y, galactose_x, c='darkblue', marker='+', s=150)
    return [im, sct_glucose, sct_galactose]

In [7]:
%%capture
# Parameters
steps = 10
# Config
fig, ax = plt.subplots(figsize=(10,10))
plt.axis('off')


In [8]:
#Run Simulation
import random 

xSize = 15
ySize= 15
board = Board(xSize,ySize)
#board.printBoard()
#board.clear()

#im = ax.imshow(board.redraw_bacteria(),  vmin=0, vmax=255)
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [board, ax])
HTML(ani.to_jshtml())


redraws!
redraws!
iteration!
[0, 1]
food1 for: 1 2 state 1
[4, 0]
[4, 4]
redraws!
iteration!
[1, 3]
[0, 3]
food11 for: 1 3 state 0
[2, 3]
[3, 2]
[3, 1]
[3, 4]
redraws!
iteration!
[1, 2]
[4, 0]
[4, 1]
[2, 5]
[5, 2]
[6, 3]
[5, 3]
redraws!
iteration!
[1, 1]
[1, 3]
[1, 6]
[6, 2]
[5, 1]
redraws!
iteration!
[0, 2]
[0, 4]
[2, 6]
[3, 2]
[1, 1]
[2, 7]
[3, 6]
[4, 3]
[3, 5]
[3, 2]
[5, 0]
[5, 1]
redraws!
iteration!
[0, 0]
[2, 3]
[2, 4]
[1, 3]
food22 for: 2 4 state 0
[4, 3]
[4, 6]
[4, 2]
[3, 7]
redraws!
iteration!
[2, 2]
[3, 3]
redraws!
iteration!
redraws!
iteration!
redraws!
